In [2]:
import nltk
# Ensure nltk punkt tokenizer is downloaded (needed to split paragraphs into sentences)
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\moura\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import pandas as pd
import numpy as np
import nltk
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# 1. Carregar o dataset e pré-processar

essays_df = pd.read_csv("C:/Users/moura/Projetos/Colab-Essays/essays.csv", encoding="latin-1")


# Dividir os ensaios em sentenças
essays_df['sentences'] = essays_df['TEXT'].apply(nltk.sent_tokenize)

# Converter 'y' e 'n' para '1' e '0'
label_columns = ['cEXT', 'cNEU', 'cAGR', 'cCON', 'cOPN']
binarizer = LabelBinarizer()
for col in label_columns:
    essays_df[col] = binarizer.fit_transform(essays_df[col])

# 2. Obter embeddings das sentenças usando SentenceTransformer

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
essays_df['embeddings'] = essays_df['sentences'].apply(lambda sentences: [model.encode(sentence) for sentence in sentences])

# 3. Agregar embeddings para representar o ensaio completo

essays_df['essay_embedding'] = essays_df['embeddings'].apply(lambda embeddings: np.mean(embeddings, axis=0))

# 4. Dividir os dados em conjuntos de treinamento e teste

X = np.array(essays_df['essay_embedding'].tolist())
y = essays_df[label_columns].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Construir e treinar um modelo LSTM

model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Reshape data for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# 6. Avaliar o modelo no conjunto de teste

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")



Vamos analisar as informações solicitadas com base no novo código fornecido:

Quais são os pré-processamentos presentes?

Tokenização de sentenças: O texto é dividido em sentenças usando nltk.sent_tokenize.
Conversão de etiquetas: As etiquetas 'y' e 'n' são convertidas para '1' e '0', respectivamente.
Obtenção de embeddings: Cada sentença do texto é convertida em um embedding usando o modelo SentenceTransformer.
Agregação de embeddings: Os embeddings de todas as sentenças de um ensaio são agregados (média) para formar um único embedding que representa o ensaio inteiro.
Qual o modelo de IA usado?

É usado um modelo de Rede Neural com camadas LSTM (Long Short-Term Memory).
O quão robusto precisa ser meu computador para executar esse código?

LSTM é menos intensivo do que modelos como BERT, mas ainda requer uma quantidade razoável de RAM e, idealmente, uma GPU para treinamento mais rápido. Uma máquina com pelo menos 8GB de RAM e uma GPU moderna seria ideal, embora seja possível executar o código em máquinas menos potentes (o treinamento seria apenas mais lento).
Quais são os otimizadores e loss para otimização?

Otimizador: Adam com uma taxa de aprendizado de 0.001.
Loss: A função de perda é binary_crossentropy, que é comumente usada para tarefas de classificação binária.
Qual é o evaluator?

O modelo é avaliado usando a métrica de acurácia (accuracy), que compara as previsões do modelo com as verdadeiras etiquetas.
Existem possíveis otimizações?

Sim, algumas possíveis otimizações incluem:
Ajustar os hiperparâmetros do modelo LSTM, como o número de neurônios, a taxa de dropout ou a taxa de aprendizado.
Experimentar outras arquiteturas de rede neural, como GRU ou combinações de LSTM e CNN.
Usar técnicas de aumento de dados para expandir o conjunto de treinamento.
Ajustar e afinar os embeddings, possivelmente treinando um modelo de embedding do zero.
Quais são as métricas mais plausíveis de se escolher para analisar esse resultado?

Acurácia: Fornece uma visão geral do desempenho do modelo.
Precisão, Recall e F1-Score: Essas métricas são particularmente importantes para avaliar o desempenho em categorias desequilibradas.
ROC e AUC: Se o modelo for adaptado para fornecer probabilidades, a curva ROC e a métrica AUC podem ser úteis para avaliar o desempenho em diferentes limiares.
Matriz de Confusão: Fornece uma visão detalhada de onde o modelo está cometendo erros.
Espero que isso ajude a esclarecer as especificações e características do código fornecido! Se você tiver mais perguntas ou precisar de mais esclarecimentos, estou à disposição.